In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import accuracy_score

In [3]:
# Загружаем модель и токенизатор
model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [27]:
# Загружаем датасет
ds = load_dataset("ai-forever/ru-reviews-classification")

In [28]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text', 'label', 'id'],
        num_rows: 45000
    })
    validation: Dataset({
        features: ['text', 'label_text', 'label', 'id'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['text', 'label_text', 'label', 'id'],
        num_rows: 15000
    })
})

In [6]:
test_data = ds["test"]

In [7]:
test_data

Dataset({
    features: ['text', 'label_text', 'label', 'id'],
    num_rows: 15000
})

# без обучения

In [8]:
# Функция для получения предсказаний модели
def predict(text):
    inputs = tokenizer(text, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).cpu().item()
    return prediction

In [9]:
# Обёртываем в tqdm и выполняем предсказания
y_true = []
y_pred = []

for row in tqdm(test_data, desc="Predicting"):
    text = row["text"]
    label = row["label"]
    y_true.append(label)
    y_pred.append(predict(text))

Predicting: 100%|██████████| 15000/15000 [05:58<00:00, 41.85it/s]


In [10]:
y_pred

[1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,


In [11]:
# Вычисляем метрику точности
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.3205


# обучим

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [3]:
import torch.nn as nn

In [6]:
model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Загружаем модель
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    ignore_mismatched_sizes=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [8]:
# Загружаем датасет
ds = load_dataset("ai-forever/ru-reviews-classification")

README.md:   0%|          | 0.00/111 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/13.3M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/4.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [14]:
set(ds['test']['label'])

{0, 1, 2}

In [15]:
# Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

In [16]:
tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text', 'label', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 45000
    })
    validation: Dataset({
        features: ['text', 'label_text', 'label', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['text', 'label_text', 'label', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15000
    })
})

In [18]:
# Удаляем ненужные столбцы
tokenized_datasets = tokenized_datasets.remove_columns(["text", "label_text", "id"])

# Указываем целевую метку
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 45000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15000
    })
})

In [20]:
# Устанавливаем формат
tokenized_datasets.set_format("torch")

In [29]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

In [31]:
# Обучение модели
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.519300,0.514424


TrainOutput(global_step=2813, training_loss=0.5031592311106498, metrics={'train_runtime': 2157.9939, 'train_samples_per_second': 20.853, 'train_steps_per_second': 1.304, 'total_flos': 2960079033600000.0, 'train_loss': 0.5031592311106498, 'epoch': 1.0})

In [32]:
# Предсказания на тестовых данных
predictions = trainer.predict(tokenized_datasets["test"])

In [33]:
predictions

PredictionOutput(predictions=array([[-1.6932026 ,  1.2117453 ,  0.6354405 ],
       [ 1.315398  ,  1.8632052 , -2.8611877 ],
       [-0.68999463,  0.7995708 , -0.1132203 ],
       ...,
       [-1.0762861 ,  1.8935097 , -0.54250526],
       [-2.187346  ,  0.83817405,  1.5670303 ],
       [-2.1591864 , -2.0107927 ,  4.587609  ]], dtype=float32), label_ids=array([0, 0, 2, ..., 1, 2, 2]), metrics={'test_loss': 0.5198866128921509, 'test_runtime': 146.1836, 'test_samples_per_second': 102.611, 'test_steps_per_second': 6.417})

In [43]:
# Преобразуем predictions.predictions в Tensor
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)

In [44]:
preds

tensor([1, 1, 1,  ..., 1, 2, 2])

In [45]:
# Оценка точности
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(predictions.label_ids, preds)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7741


In [48]:
# Получаем id из тестового набора и предсказанные метки
test_ids = ds["test"]["id"]  # Используем id из оригинального датасета
pred_labels = preds.numpy()  # Преобразуем предсказания в numpy для совместимости с pandas

In [49]:
test_ids

['17467',
 '15189',
 '69828',
 '14405',
 '62334',
 '68990',
 '7339',
 '10934',
 '63494',
 '68248',
 '33489',
 '39655',
 '37288',
 '69839',
 '7033',
 '60348',
 '6706',
 '2523',
 '82619',
 '18543',
 '42837',
 '47587',
 '22425',
 '10675',
 '17777',
 '48692',
 '75254',
 '76309',
 '16063',
 '69137',
 '13951',
 '48838',
 '49921',
 '69414',
 '49142',
 '60066',
 '71126',
 '59761',
 '71630',
 '47028',
 '48602',
 '13329',
 '3975',
 '43386',
 '63708',
 '53244',
 '30476',
 '9552',
 '76453',
 '24766',
 '20754',
 '49549',
 '65095',
 '14113',
 '31315',
 '44359',
 '30234',
 '77372',
 '13648',
 '73617',
 '29457',
 '71913',
 '21118',
 '5816',
 '8409',
 '38096',
 '17234',
 '4451',
 '11967',
 '15244',
 '32686',
 '24217',
 '82409',
 '40415',
 '74425',
 '35417',
 '49525',
 '11770',
 '44735',
 '68123',
 '12537',
 '72745',
 '44163',
 '76086',
 '82801',
 '61270',
 '6676',
 '63858',
 '45491',
 '45412',
 '14616',
 '13850',
 '44998',
 '29901',
 '35657',
 '789',
 '23413',
 '68385',
 '19750',
 '47152',
 '19219',
 '

In [51]:
import pandas as pd

In [52]:
# Создаем DataFrame для записи результатов
submission_df = pd.DataFrame({
    "id": test_ids,
    "label": pred_labels
})

In [53]:
submission_df

,id,label
0,17467,1
1,15189,1
2,69828,1
3,14405,0
4,62334,2
...,...,...
14995,44321,1
14996,71046,2
14997,48005,1
14998,75150,2


In [ ]:
# Сохраняем DataFrame в CSV файл
submission_df.to_csv("submission.csv", index=False)